In [ ]:
import os
import pandas as pd
import numpy as np
%matplotlib inline 

In [ ]:
def score_ys(data,p,middle_score,outlier_pct,score_):
    score_cutoff=np.mean(data[p]) #平均预测概率，假设平均预测概率=实际坏账率
    score_cutoff=np.log(score_cutoff/(1-score_cutoff)) #平均预测概率对应的logodds值
    middle_score=middle_score #可比分数
    outlier_pct = outlier_pct #极端值临界点
    data['odd']=np.log(data[p]/(1-data[p])) #logodds值
    max_odds =np.percentile(data['odd'],100-outlier_pct)#99分位点对应的logodds值
    min_odds=np.percentile(data['odd'],outlier_pct)#1分位点对应的logodds值
    max_p=np.percentile(data[p],100-outlier_pct)#99分位点对应的P值
    min_p=np.percentile(data[p],outlier_pct)#1分位点对应的p值
    ##第一段公式
    b1 =(900-850)/(0-min_p)
    l1 = b1*data[p] + 900
    ##第二段公式
    b2 = (850-middle_score)/(min_odds -score_cutoff )
    a2 = middle_score - b2 * score_cutoff
    l2 = a2 + b2 * data['odd']
    ##第三段公式
    b3 = (middle_score-350)/(score_cutoff -max_odds )
    a3 = middle_score - b3 * score_cutoff
    l3 = a3 + b3 * data['odd']
    ##第四段公式
    b4 = (350-300)/(max_p-1)
    a4 = 300-b4
    l4 = a4+b4*data[p]
    data[score_]=np.where(data['odd']<=score_cutoff,l2,l3)
    data[score_]=np.where(data['odd']> max_odds,l4,data[score_])
    data[score_]=np.where(data['odd']<min_odds ,l1,data[score_])
    data[score_].hist(bins=30)#分数的直方图
    
    x1 = score[(score.odd <= min_odds)]
    x2 = score[(score.odd > min_odds) & (score.odd<=score_cutoff)]
    x3 =  score[(score.odd > score_cutoff) & (score.odd<=max_odds)]
    x4 = score[(score.odd > max_odds)]
    
    fig = plt.figure(figsize=(12,8))
    #ax1 = fig.add_subplot(111)
    plt.plot(x1.phat,x1.score_,'or--',label = u'l1')
    plt.plot(x2.phat,x2.score_,'bs',label = u'l2')
    plt.plot(x3.phat,x3.score_,'gs',label = u'l3')
    plt.plot(x4.phat,x4.score_,'oy-',label = u'l4')
    plt.legend()